# Extraindo dados de vagas do Linkedin através de Web Scraping

### Instalando as bibliotecas necessárias

In [ ]:
!pip install requests # Usada para fazer requisições HTTP, ou seja, acessar páginas da web e pegar seu conteúdo.
!pip install beautifulsoup4 # Usada para processar e interpretar o código HTML das páginas da web.

### Importando as Bibliotecas

In [31]:
# Importando as bibliotecas necessárias

import requests  # Para acessar páginas da web
from bs4 import BeautifulSoup  # Para extrair informações do código HTML das páginas
import pandas as pd  # Para criar e manipular tabelas de dados

### Coletando informações de filtros para a Vaga

In [32]:
nome_vaga = input("Digite o nome da vaga: ")
nome_vaga_tratada = nome_vaga.replace(" ", "%20").lower()

In [33]:
experiencia = input("Digite o tipo de experiência: (0 = Todos, 1 = Estágio, 2 = Assistente, 3 = Júnior, 4 = Pleno-Sênior e 5 = Diretor)")
if experiencia == "0" or int(experiencia) > 5:
    experiencia_tratada = ''
else:
    experiencia_tratada = f'''&f_E={experiencia}'''

In [34]:
tipo_vaga = input("Digite o tipo da vaga: (0 = Todos, 1 = Presencial, 2 = Remoto, 3 = Híbrido)")
if tipo_vaga == "0" or int(tipo_vaga) > 3:
    tipo_vaga_tratada = ''
else:
    tipo_vaga_tratada = f'''&f_WT={tipo_vaga}'''

### Criando a URL de busca

In [35]:
base_url = f'''
https://www.linkedin.com/jobs/search?
keywords={nome_vaga_tratada}
&location=Brazil
&geoId=106057199
{experiencia_tratada}
&f_TPR=r86400
{tipo_vaga_tratada}
&position=1
&pageNum=0 
'''

In [36]:
base_url

'\nhttps://www.linkedin.com/jobs/search?\nkeywords=analista%20de%20dados\n&location=Brazil\n&geoId=106057199\n&f_E=2\n&f_TPR=r86400\n&f_WT=2\n&position=1\n&pageNum=0 \n'

In [37]:
base_url = base_url.replace("\n", "")

In [38]:
base_url

'https://www.linkedin.com/jobs/search?keywords=analista%20de%20dados&location=Brazil&geoId=106057199&f_E=2&f_TPR=r86400&f_WT=2&position=1&pageNum=0 '

### Fazendo a requisição para o Linkedin

In [39]:
response = requests.get(base_url)

In [40]:
print(response)

<Response [200]>


### Extraindo os dados da Página

In [41]:
site = BeautifulSoup(response.text, 'html.parser')

In [ ]:
site

### Encontrando os elementos com informações das vagas

In [42]:
dados = site.find_all("div", attrs={"class" : "base-card"})

### Criando uma lista para armazenar os dados

In [43]:
armazenando_vaga = []
armazenamento_empresa = []
armazenamento_localizacao = []
armazenamento_tempo = []
armazenamento_data = []
armazenamento_link = []

### Extraindo informações de cada Vaga

In [45]:


for palavra in dados:
  #Nome da vaga
  nome_vaga = palavra.find_all("h3", attrs = {"class": "base-search-card__title"})[0].text.strip()
  armazenando_vaga.append(nome_vaga)


  #nome empresa
  nome_empresa = palavra.find_all("h4", attrs = {"class": "base-search-card__subtitle"})[0].text.strip()
  armazenamento_empresa.append(nome_empresa)


  #Localização

  localizacao = palavra.find_all("span", attrs = {"class": "job-search-card__location"})[0].text.strip()
  armazenamento_localizacao.append(localizacao)

  #Tempo disponibilidade da vaga
  tempo_vaga = palavra.find_all("time")[0].text.strip()
  armazenamento_tempo.append(tempo_vaga)

  #data 
  data = palavra.find_all("time", attrs = {"class": "job-search-card__listdate--new"})[0]
  data_trat = data["datetime"]
  armazenamento_data.append(data_trat)

  # se quiser em uma linha só: data_trat = palavra.find_all("time")[0].get('datetime')
  # ou data_trat = palavra.find_all("time")[0]["datetime"]

  #Link da vaga 
  link_vaga = palavra.find_all("a", attrs = {"class": "base-card__full-link"})[0].get('href')
  armazenamento_link.append(link_vaga)




### Criando um dicionário com os dados coletados


In [46]:
base_vagas = {
"Vaga" : armazenando_vaga,
"Empresa": armazenamento_empresa,
"Localizacao" : armazenamento_localizacao,
"Tempo_postada": armazenamento_tempo,
"Data da Vaga" : armazenamento_data,
"Link da Vaga" : armazenamento_link
}

### Criando um dataframe para armazenar os dados

In [47]:
df = pd.DataFrame(base_vagas)

In [48]:
df

,Vaga,Empresa,Localizacao,Tempo_postada,Data da Vaga,Link da Vaga
0,Desenvolvedor SQL Server (T-SQL),Grupo Adriano Cobuccio,"Monte Belo, Minas Gerais, Brazil",9 hours ago,2025-02-07,https://br.linkedin.com/jobs/view/desenvolvedo...
1,Engenheiro de dados junior,Netvagas,"São Paulo, São Paulo, Brazil",15 hours ago,2025-02-07,https://br.linkedin.com/jobs/view/engenheiro-d...
2,Research Assistant,Alana AI,Brazil,2 hours ago,2025-02-07,https://br.linkedin.com/jobs/view/research-ass...
3,Analista de Desenvolvimento - Pleno,Implanta IT Solutions,"Goiânia, Goiás, Brazil",21 hours ago,2025-02-06,https://br.linkedin.com/jobs/view/analista-de-...
4,ANALISTA DE DESENVOLVIMENTO II,G4F,"Salvador, Bahia, Brazil",19 hours ago,2025-02-06,https://br.linkedin.com/jobs/view/analista-de-...
...,...,...,...,...,...,...
111,Data Scientist/Analyst,Turing,"Porto Alegre, Rio Grande do Sul, Brazil",22 hours ago,2025-02-06,https://br.linkedin.com/jobs/view/data-scienti...
112,Data Scientist/Analyst,Turing,"São Paulo, São Paulo, Brazil",22 hours ago,2025-02-06,https://br.linkedin.com/jobs/view/data-scienti...
113,Data Scientist/Analyst,Turing,Brazil,22 hours ago,2025-02-06,https://br.linkedin.com/jobs/view/data-scienti...
114,Data Scientist/Analyst,Turing,Brazil,22 hours ago,2025-02-06,https://br.linkedin.com/jobs/view/data-scienti...


### Proximos passos, Fazer a inclusão dos daos em um banco de dados e subir para a AWS